<a href="https://colab.research.google.com/github/LogicPenguins/MachineLearningProjects/blob/master/DDNClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

First lets setup some pre-requisities such as library and module imports, as well as reading in our csv data and initializing some of our testing data structures.

In [8]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

CSV_COLUMNS = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']


train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMNS, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMNS, header=0)

train_y = train.pop("Species")
test_y = test.pop("Species")


In [15]:
def input_fn(features, labels, training=True, batch_size=256):
    # Model needs dataset object below for processing 
    # Features are the raw data and labels are the y-vals
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training: 
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[30, 10],
    n_classes=3
)


In [16]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000
)

86% Accuracy. Not bad!

In [17]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False)
)

print(f"\nAccuracy: {round((100*eval_result['accuracy']), 2)}%")


Accuracy: 86.67%


Now we can make predictions using some simple input code to actually make use of our trained model! 

Since the input function below is only going to be used for predictions, we don't need to feed y values to it since y values only serve to train the model!

In [25]:
def input_fn_pred(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print('Please type number values as prompted')
for feature in features:
    while True:
        val = input(feature + ": ")
        if not val.isdigit(): 
            break
        
    predict[feature] = [float(val)]

    
        

# The classifier requires an actual function to be inputted, so we need to 
# create an anonymous function (aka a lambda) to be able tl pass in the input_fn
predictions = classifier.predict(input_fn=lambda: input_fn_pred(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(f'Prediction is {SPECIES[class_id]} {round(100 * probability,2)}%')

Please type number values as prompted
SepalLength: 3.4
SepalWidth: 4.5
PetalLength: 3.4
PetalWidth: 6.4
Prediction is Virginica 98.58


After a bit of debugging, it finally works!